# Обучение сети для извлечения дескрипторов лица
## Реализация InsightFace

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
keras.mixed_precision.set_global_policy("mixed_float16")

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  GeForce GTX 1060 6GB, compute capability 6.1
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [3]:
# Подготовим наш датасет для обучения
# Данные в реализации Insightface хранятся в формате MXNet Record из которых необходимо извлечь картинки для обучения
def MXnet_record_to_folder(dataset_dir, save_dir=None):
    import os
    import numpy as np
    import mxnet as mx
    from tqdm import tqdm

    if save_dir == None:
        save_dir = (dataset_dir[:-1] if dataset_dir.endswith("/") else dataset_dir) + "_112x112_folders"
    idx_path = os.path.join(dataset_dir, "train.idx")
    bin_path = os.path.join(dataset_dir, "train.rec")

    print("save_dir = %s, idx_path = %s, bin_path = %s" % (save_dir, idx_path, bin_path))
    if os.path.exists(save_dir):
        print("%s already exists." % save_dir)
        return

    imgrec = mx.recordio.MXIndexedRecordIO(idx_path, bin_path, "r")
    rec_header, _ = mx.recordio.unpack(imgrec.read_idx(0))

    for ii in tqdm(range(1, int(rec_header.label[0]))):
        img_info = imgrec.read_idx(ii)
        header, img = mx.recordio.unpack(img_info)
        # img_idx = str(int(np.sum(header.label)))
        img_idx = str(int(header.label if isinstance(header.label, float) else header.label[0]))
        img_save_dir = os.path.join(save_dir, img_idx)
        if not os.path.exists(img_save_dir):
            os.makedirs(img_save_dir)
        with open(os.path.join(img_save_dir, str(ii) + ".jpg"), "wb") as ff:
            ff.write(img)


def MXnet_bin_files_to_tf(test_bins, limit=0):
    import io
    import pickle
    import tensorflow as tf
    from skimage.io import imread

    print("test_bins =", test_bins)
    for test_bin_file in test_bins:
        with open(test_bin_file, "rb") as ff:
            bins, issame_list = pickle.load(ff, encoding="bytes")

        bb = [bytes(ii) for ii in bins[: limit * 2] + bins[-limit * 2 :]]
        print("Saving to %s" % test_bin_file)
        with open(test_bin_file, "wb") as ff:
            pickle.dump([bb, issame_list[:limit] + issame_list[-limit:]], ff)

In [4]:
# Пути к датасету
DATASET_DIR = '/app/data/faces_emore'
EVAL_BINS = [DATASET_DIR+'/lfw.bin', DATASET_DIR+'/cfp_fp.bin', DATASET_DIR+'/agedb_30.bin']

# Извлекаем картинки
MXnet_record_to_folder(DATASET_DIR)

# Конвертим bin файлы для работы с TF
MXnet_bin_files_to_tf(EVAL_BINS)

save_dir = /app/data/faces_emore_112x112_folders, idx_path = /app/data/faces_emore/train.idx, bin_path = /app/data/faces_emore/train.rec
/app/data/faces_emore_112x112_folders already exists.
test_bins = ['/app/data/faces_emore/lfw.bin', '/app/data/faces_emore/cfp_fp.bin', '/app/data/faces_emore/agedb_30.bin']
Saving to /app/data/faces_emore/lfw.bin
Saving to /app/data/faces_emore/cfp_fp.bin
Saving to /app/data/faces_emore/agedb_30.bin


In [5]:
from tensorflow import keras
import losses, train, models
import tensorflow_addons as tfa

basic_model = models.buildin_models("r100", dropout=0, emb_shape=512, output_layer="GDC")
data_path = '/app/data/faces_emore_112x112_folders'

tt = train.Train(data_path, save_path='keras_resnet100_emore.h5', eval_paths=EVAL_BINS,
                basic_model=basic_model, batch_size=32, random_status=0,
                lr_base=0.1, lr_decay=0.5, lr_decay_steps=16, lr_min=1e-5)

optimizer = tfa.optimizers.SGDW(learning_rate=0.1, momentum=0.9, weight_decay=5e-5)
sch = [
  {"loss": losses.ArcfaceLoss(scale=16), "epoch": 5, "optimizer": optimizer},
  {"loss": losses.ArcfaceLoss(scale=32), "epoch": 5},
  {"loss": losses.ArcfaceLoss(scale=64), "epoch": 40},
]

tt.train(sch, 0)

>>>> L2 regularizer value from basic_model: 0
>>>> Init type by loss function name...
>>>> Train arcface...
>>>> Init softmax dataset...
>>>> reloaded from dataset backup: faces_emore_112x112_folders_shuffle.npz
>>>> Image length: 5822653, Image class length: 5822653, classes: 85742
>>>> Use specified optimizer: <tensorflow_addons.optimizers.weight_decay_optimizers.SGDW object at 0x7fdc6284e978>
>>>> Insert weight decay callback...
>>>> Add arcface layer, loss_top_k=1...
>>>> loss_weights: {'arcface': 1}
Epoch 1/5

Learning rate for iter 1 is 0.10000000149011612
Weight decay is 4.999999873689376e-05
   105/181957 [..............................] - ETA: 34:09:06 - loss: 19.6046 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
# Обучение на 1060GTX 50 эпох займет 70 дней...